```
p_паттерн парсит текст (str, pos), 
    вызывая другие паттерны, возвращающие древовидные структуры
    эти древовидные структуры передает одному из правил
        (остальные вызовы правил закомментированы 
            - в рамках одного текста каждый паттерн имеет ровно 1 смысл)
    и возвращает (pos, результат этого правила), помещенный в массив
    
    /*пока на ошибках парсинга концентрировать не будем*/
    если ничего не удалось распарсить, ???возвращаемый массив будет пустым???
    если удалось распарсить несколько вариантов - в массиве будет насколько вариантов
        сначала парсятся все исключения
        потом парсятся все обычные варианты (если нет исключений)
```

```
r_правило получает список древовидных структур
    обрабатывает их по определенному правилу
    возвращает древовидную структуру
```

```
древовидная структура - map, со следующими ключами
    type: 'noun'/'adj'/'verb'/'numeric'/...
    постоянные параметры (для сущ.: род, число)
    переменные параметры (для сущ.: падеж)
    talk: массив древовидных структур
        или пар (тип, слово), где тип - main/punct/other
```

```
str(древовидная структура)
    превращает древовидную структуру в строку
```

```
для каждого типа элемента древовидной структуры будет map
    где каждой строке (для сущ. - слово в и.п.) будет соответствовать
        структура данных, которая вместе в переменными параметрами передается в 
        sh_функция для этого типа элемента древовидной структуры
            которая будет возвращать слово в соотв. форме (для сущ. в соотв. падеже)
```

# Общее

In [1]:
def throw(ex):
    raise ex

In [2]:
from copy import deepcopy

In [3]:
class S(str): # строка с атрибутом
    __slots__='attrs'
#    def __new__(self, *args, **kwargs):
#        return super(S, self).__new__(self, *args, **kwargs)
    def __init__(self,*args,**kwargs):
#        super(S, self).__init__()
        self.attrs=dict()

def attrs2right(l,r):
    return r
def attr2left(l,r):
    return l
#.pre - строка - конкатенируется (? с пробелом?)
#.post - строка - конкатенируется (? с пробелом?)
#.prepost - массив пар строк - конкатенируется
#.change - функция - если не совпадают - ошибка

In [4]:
class TextError(ValueError):
    pass

In [5]:
def en2ru(s): # main
    s=[ i for i in tokenizer(s)]
    # добавить дочитывание точки и остаточных пробелов
    rezs=[res for pos,res in p_noun(s,0) if pos==len(s)]
    if len(rezs)>0: return [str(i) for i in rezs]
    rezs=[res for pos,res in p_adj(s,0) if pos==len(s)]
    if len(rezs)>0: return [str(i) for i in rezs]
    return []

## Паттерны парсинга

In [6]:
def debug_pp(fun):
    def wrapper(s,p):
        print('{'+' '*p,fun.__name__)
        rezs=fun(s,p)
        for (p,r) in rezs:
            assert isinstance(r,Struct)
            if r.type=='num' : assert isinstance(r,StNum)
        print('.'+' '*p,len(rezs),' in ',fun.__name__,'}',[str(r) for (p,r) in rezs])
        return rezs
#    return wrapper
    return fun

In [7]:
def sp_spcs(str,pos):
    pos1=pos
    while pos1<len(str) and (str[pos1]==' ') : #  or str[pos1]=='\n' or str[pos1]=='\t'
        pos1+=1
    return [] if pos1==pos else [(pos1,'')]

In [8]:
def sp_word(str,pos,word=None):
    if word!=None:
        return [(pos+len(word),word)] if str[pos:pos+len(word)]==word else []
    pos1=pos
    while pos1<len(str) and \
        (str[pos1]>='a' and str[pos1]<='z' or str[pos1]>='A' and str[pos1]<='Z') :
        pos1+=1
    return [] if pos1==pos else [(pos1,str[pos:pos1])]

In [9]:
def sp_punct(str,pos):
    pos1=pos
    while pos1<len(str) and str[pos1]!=' ' and\
        not (str[pos1]>='a' and str[pos1]<='z' or str[pos1]>='A' and str[pos1]<='Z') :
        pos1+=1
    return [] if pos1==pos else [(pos1,str[pos:pos1])]    

In [10]:
def tokenizer(s):
    pos=0
    while pos<len(s):
        tmp=sp_word(s,pos)
        if len(tmp)>0:
            yield S(tmp[0][1]) # можем добавить атрибуты
            pos=tmp[0][0]
        tmp=sp_spcs(s,pos)
        if len(tmp)>0:
            pos=tmp[0][0]
        tmp=sp_punct(s,pos)
        if len(tmp)>0:
            yield S(tmp[0][1]) # можем добавить атрибуты
            pos=tmp[0][0]

In [11]:
def sp_seq(str,pos,patterns):
#    for patt in patterns:
#        tmp=patt(str,pos)
#        if len(tmp)>1 : raise NotImplementedError()
#        if len(tmp)==0 : return []
#        (pos,tmp)=tmp[0]
#        rezs.append(tmp)
#    return [(pos,rezs)]
    if len(patterns)==1:
        return [(p,[r]) for (p,r) in patterns[0](str,pos)]
    first=patterns[0](str,pos)
    first.sort(key=lambda i:i[0]) # в дальнейшем отключить повторное вычисление 
    # продолжения для одинаковых позиций
    rezs=[]
    for r in first:
        tmp=sp_seq(str,r[0],patterns[1:])
        for rr in tmp:
            rr[1].insert(0,r[1])
        rezs+=tmp
    return rezs

In [12]:
#объекты из словаря и паттернов копируются (полностью), потом из них строится дерево
#в эти копии потом добваляется .attrs
def word(w):
    assert type(w)==str
    from copy import copy
    return lambda s,p: [(p+1,copy(s[p]))] if p<len(s) and s[p]==w else []

def fromdict(d):
    assert type(d)==dict
    def p_fromdict(s,p):
        if p<len(s) and s[p] in d:
            tmp=deepcopy(d[s[p]])
            tmp.attrs=s[p].attrs
            return [(p+1,tmp)]
        else:
            return []
    return p_fromdict

def p_seq(s,p,patterns,handler,numbers=None):
# p_seq(s,p,[p0,p1,p2,p3],handle,[0,2,3]) ->
# for k:
#   (pos1,rez1) = sp_seq(s,p,[p0,p1,p2,p3]) [k]
#   rezs.append((pos1,handle(rezs[0],rezs[2],rezs[3],)))
    if numbers==None : numbers=range(len(patterns))
    rezs=[]
    for (pos1,rez1) in sp_seq(s,p,patterns):
        m=[False for i in range(len(patterns))]
        for i in numbers: m[i]=True
        for i in range(max(numbers)):
            if not m[i]: attrs2right(rez1[i],rez1[i+1])
        for i in range(len(patterns)-1,max(numbers),-1):
            if not m[i]: attrs2left(rez1[i-1],rez1[i])
        rezs.append((pos1,handler(*[rez1[i] for i in numbers])))
    return rezs

##### тесты

In [13]:
[i for i in range(5,3,-1)]

[5, 4]

In [14]:
def h(*args):
    print(args)
p_seq(['1','2','3'],0,[word('1'),word('2'),word('3')],h,[0,2])

('1', '3')


[(3, None)]

In [15]:
s=S('qwe')
s.attrs=8
s=='qwe'

True

In [16]:
s=S('qwe')
s.attrs

{}

In [17]:
from copy import copy
s=S('qwe')
copy(s)

'qwe'

In [18]:
s=S('qwe')
repr(s)

"'qwe'"

# Словарь

## Классы отображения

In [19]:
class Struct:
    type=None
    talk=[] #массив структур или пар (строка-тип,строка-значение)
    def __str__(self):
        raise NotImplementedError('virtual function')

In [20]:
class StNoun(Struct):
    def __init__(self,odush=None,rod=None,chis=None,pad=None,word=None,och=None):
        self.type='noun'
        
        if odush==None and rod==None and chis==None and pad==None and word==None and och==None:
            self.odush=False
            self.rod='m'
            self._chis='ed'
            self._pad='ip'
            self.och=0 # составное существительное
            self.talk=[]
        else:
            if type(odush)!=bool : raise TypeError('odush must be bool')
            self.odush=odush

            if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
            self.rod=rod

            if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
            self._chis=chis

            if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp' and pad!='pp' : 
                raise TypeError('pad must be ip, rp, dp, vp, tp or pp')
            self._pad=pad
            
            if type(och)!=str and och!=0 and Och!=None:
                raise TypeError('och must be string or 0 or None')
            self.och=och

            self.word=word
            self.talk=[('main',word)] # в массиве служит лишь меткой, word значения не имеет
            
    pad=property()
    @pad.getter
    def pad(self):
        return self._pad
    @pad.setter
    def pad(self,val):
        self._pad=val
        for i in self.talk:
            if isinstance(i,StNoun) or isinstance(i,StAdj):
                i.pad=val
        
    chis=property()
    @chis.getter
    def chis(self):
        return self._chis
    @chis.setter
    def chis(self,val):
        #print('change chis')
        if self._chis!=val:
            if self.och==None :
                raise RuntimeError(self.word+' has no och')
            elif type(self.och)==str:
                tmp=self.och
                self.och=self.word
                self.word=tmp
            elif self.och==0:
                pass
            else:
                raise RuntimeError('internal error: StNoun.och = '+self.och)
            self._chis=val
            for i in self.talk:
                if isinstance(i,StNoun) or isinstance(i,StAdj):
                    i.chis=val
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',word:'+self.word+',och:'+self.och+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(\
            str(i) if isinstance(i,Struct) else \
            show_noun_map[self.word](self) if i[0]=='main' else\
            i[1] if i[0]=='other' else\
            '#'+i[1] if i[0]=='punct' else throw(RuntimeError()) \
                        for i in self.talk).replace(' #','')

def show_noun1(pad,word,ip,rp,dp,vp,tp,pp):
    if pad=='ip' : return word+ip
    if pad=='rp' : return word+rp
    if pad=='dp' : return word+dp
    if pad=='vp' : return word+vp
    if pad=='tp' : return word+tp
    if pad=='pp' : return word+pp
    raise InternalError()


In [21]:
class StNum(Struct):
    def __init__(self,odush=None,rod=None,chis=None,pad=None,word=None,quantity=None):
        self.type='num'
        
        if odush==None and rod==None and chis==None and pad==None and word==None :
            self.odush=False
            self.rod='m'
            self.chis='mn'
            self._pad='ip'
            self.talk=[]
        else:
            if type(odush)!=bool : raise TypeError('odush must be bool')
            self.odush=odush

            if rod!='m' and rod!='g' and rod!='s' :raise TypeError('rod must be m or g or s')
            self.rod=rod

            if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
            self.chis=chis

            if pad!='ip' and pad!='rp' and pad!='dp' and \
                    pad!='vp' and pad!='tp' and pad!='pp' : 
                raise TypeError('pad must be ip, rp, dp, vp, tp or pp')
            self._pad=pad
            
            if quantity!='2-4' and quantity!='1' and quantity!='>=5':
                raise TypeError('quantity must be "1", "2-4" or ">=5"')
            self.quantity=quantity

            self.talk=[('main',word)]
            
    pad=property()
    @pad.getter
    def pad(self):
        return self._pad
    @pad.setter
    def pad(self,val): #...
        #print('num.pad',val)
        self._pad=val
        for i in self.talk:
            if isinstance(i,StNoun) or isinstance(i,StAdj):
                if (val=='ip' or val=='vp' and not i.odush) and self.quantity!='1':
                    if self.quantity=='2-4' and i.rod=='m':
                        i.chis='ed'
                        i.pad='rp'
                #    else:
                #        i.pad='rp'
                else:
                    #print(val)
                    i.pad=val
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(\
            str(i) if isinstance(i,Struct) else \
            show_num_map[i[1]](self) if i[0]=='main' else\
            i[1] if i[0]=='other' else\
            '#'+i[1] if i[0]=='punct' else throw(RuntimeError()) \
                        for i in self.talk).replace(' #','')

def show_num1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    else: raise InternalError()
        
    return word+rez


In [22]:
class StAdj(Struct):
    def __init__(self,odush=None,rod=None,chis=None,pad=None,word=None):
        self.type='adj'
        
        if odush==None and rod==None and chis==None and pad==None and word==None :
            self.odush=False
            self.rod='m'
            self.chis='ed'
            self.pad='ip'
            self.talk=[]
        else:
            if type(odush)!=bool : raise TypeError('odush must be bool')
            self.odush=odush

            if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
            self.rod=rod

            if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
            self.chis=chis

            if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp'\
                    and pad!='pp'  and pad!='sh' : 
                raise TypeError('pad must be ip, rp, dp, vp, tp, pp or sh')
            self.pad=pad

            self.talk=[('main',word)]
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(\
            str(i) if isinstance(i,Struct) else \
            show_adj_map[i[1]](self) if i[0]=='main' else\
            i[1] if i[0]=='other' else\
            '#'+i[1] if i[0]=='punct' else throw(RuntimeError()) \
                        for i in self.talk).replace(' #','')

def show_adj1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp,sh)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp,sh)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    elif st.pad=='sh' : rez=sh
    else: raise InternalError()
        
    return word+rez


## Отображение

In [23]:
show_noun_map={
}

In [24]:
show_adj_map={
    'летучий'  :lambda st: show_adj1(st,'летуч' ,
        {
            'm' :('ий','его','ему',('его','ий'),'им' ,'ем','' ),
            's' :('ее','его','ему',('ий', 'ий'),'им' ,'ем','е'),
            'g' :('ая','ей' ,'ей' ,('ую', 'ую'),'ей' ,'ей','а'),
            'mn':('ие','их' ,'им' ,('их', 'ие'),'ими','их','и'),
        }
    ) ,
}

In [25]:
show_num_map={
    'один'   :lambda st: show_num1(st,'од' ,
        {
            'm' :('ин','ного','ному',('ного','ин'),'ним' ,'ном'),
            's' :('но','ного','ному',('но'  ,'но'),'ним' ,'ном'),
            'g' :('на','ной' ,'ной' ,('ну'  ,'ну'),'ной' ,'ной'),
            'mn':('ни','них' ,'ним' ,('них', 'ни'),'ними','них'),
        }
    ) ,
    'три'   :lambda st: show_num1(st,'тр' ,
        {
            'mn':('и','ёх' ,'ём' ,('ёх', 'и'),'емя','ёх'),
        }
    ) ,
}

## Правила

In [26]:
ruwords={} #какие-то однословные правила складываются сюда, а какие-то остаются в функциях

def add_runoun(name,och,odush,rod,chis,word,ip,rp,dp,vp,tp,pp):
    ruwords[name]=StNoun(odush,rod,chis,'ip',name,och)
    show_noun_map[name]=lambda st: show_noun1(st.pad,word,ip,rp,dp,vp,tp,pp)

def add_runoun2(name,rod,word,ip,rp,dp,vp,tp,pp,\
               mname,odush,mword,mip,mrp,mdp,mvp,mtp,mpp):
    add_runoun(name,mname,odush,rod,'ed',word,ip,rp,dp,vp,tp,pp)
    add_runoun(mname,name,odush,rod,'mn',mword,mip,mrp,mdp,mvp,mtp,mpp)

In [27]:
#                                  ип ,нет ,дать,вижу,творю,думаю
add_runoun2("кот"   ,'m'  ,'кот'  ,'' ,'а' ,'у' ,'а' ,'ом' ,'е' ,
            "коты"  ,True ,'кот'  ,'ы','ов','ам','ов','ами','ах')

add_runoun2("урок"  ,'m'  ,'урок' ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "уроки" ,False,'урок' ,'и','ов','ам','и' ,'ами','ах')

add_runoun2("кошка" ,'g'  ,'кошк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "кошки" ,True ,'кош'  ,'ки','ек','кам','ек','ками','ках')

add_runoun2("собака",'g'  ,'собак','а' ,'и' ,'е'  ,'у'  ,'ой' ,'е' ,
            "собаки",True ,'собак','и' ,''  ,'ам' ,''   ,'ами','ах')

add_runoun2("ручка" ,'g'  ,'ручк' ,'а' ,'и' ,'е'  ,'у'  ,'ой' ,'е'  ,
            "ручки" ,False,'руч'  ,'ки','ек','кам','ки','ками','ках')
add_runoun2("чашка" ,'g'  ,'чашк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "чашки" ,False,'чаш'  ,'ки','ек','кам','ки','ками','ках')

add_runoun2("кепка" ,'g'  ,'кепк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "кепки" ,False,'кеп'  ,'ки','ок','кам','ки','ками','ках')
add_runoun2("шапка" ,'g'  ,'шапк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "шапки" ,False,'шап'  ,'ки','ок','кам','ки','ками','ках')

add_runoun2("крыса" ,'g'  ,'крыс' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "крысы" ,True ,'крыс' ,'ы','' ,'ам',''  ,'ами','ах')
add_runoun2("курица",'g'  ,'куриц','а','ы','е' ,'у' ,'ой' ,'е' ,
            "курицы",True ,'куриц','ы','' ,'ам',''  ,'ами','ах')
	    
add_runoun2("шляпа" ,'g'  ,'шляп' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "шляпы" ,False,'шляп' ,'ы','' ,'ам','ы' ,'ами','ах')

add_runoun2("свинья",'g'  ,'свинь','я' ,'и' ,'е'  ,'ю' ,'ёй'  ,'е'  ,
            "свиньи",True ,'свин' ,'ьи','ей','ьям','ей','ьями','ьях')

add_runoun2("ружьё" ,'s'  ,'ружь' ,'ё' ,'я','ю' ,'ё' ,'ём' ,'е' ,
            "ружья" ,False,'руж'  ,'ья','ей','ьям','ьи','ьями','ьях')

add_runoun2("мышь"  ,'g'  ,'мыш'  ,'ь','и' ,'и' ,'ь' ,'ью' ,'и' ,
            "мыши"  ,True ,'мыш'  ,'и','ей','ам','ей','ами','ах')


In [28]:
def r_letuchij(): return StAdj(True,'g','ed','ip','летучий')

def r_odin(): return StNum(False,'m','ed','ip','один','1')
def r_tri(): return StNum(False,'m','mn','ip','три','2-4')

In [29]:
def r_noun_and_noun(sn,n):
    root=StNoun()
    root.chis='mn'
    root.och=None # disable change root.chis
    root.talk=[sn,('other','и'),n]
    return root
def r_noun_comma_noun(sn,n):
    root=StNoun()
    root.chis='mn'
    root.och=None # disable change root.chis
    root.talk=[sn,('punct',','),n]
    return root

def r_a_noun(a,n): return attrs2right(a,n)
def r_adj_noun(a,n): 
    a.rod=n.rod
    a.odush=n.odush
    a.chis=n.chis
    n.talk.insert(0,a)
    return n

def r_numeral_noun(num,n):
    if num.chis!=n.chis :
        print('не совпадают числа числ. и сущ.:',str(num),str(n))
    num.rod=n.rod
    num.odush=n.odush
    num.talk.append(n)
    num.pad='ip' # setup n.pad
    return num

def r_noun_numeral(n,num):
    n.talk.append(num)
    return n

## Словарь

In [30]:
dict_adj={
    'a' : S(''),
    #'a' : r_nekotoryj(),
    'an' : S(''),
    #'an' : r_nekotoryj(),
}

In [31]:
dict_noun={
    'cat':   ruwords["кошка"],
    #'cat':   ruwords["кот"],
    'cats':   ruwords["кошки"],
    #'cats':   ruwords["коты"],
    
    #'rat':   ruwords["мышь"],
    'rat':   ruwords['крыса'],
    #'rats':   ruwords["мыши"],
    'rats':   ruwords['крысы'],
    
    'bat':   r_adj_noun(deepcopy(r_letuchij()),deepcopy(ruwords["мышь"])),
    'bats':   r_adj_noun(deepcopy(r_letuchij()),deepcopy(ruwords["мыши"])),
    
    'lesson':ruwords["урок"],
    'lessons':ruwords["уроки"],
    
    'cap':   ruwords["кепка"],
    #'cap':  ruwords["шапка"],
    'caps':   ruwords["кепки"],
    #'caps':  ruwords["шапки"],
    
    'pen':   ruwords["ручка"],
    'pens':   ruwords["ручки"],
    
    'hat':   ruwords["шляпа"],
    'hats':   ruwords["шляпы"],
    
    'hen':   ruwords["курица"],
    'hens':   ruwords["курицы"],
    
    'dog':   ruwords['собака'],
    'dogs':   ruwords['собаки'],
    'pig':   ruwords['свинья'],
    'pigs':   ruwords['свиньи'],
    'gun':   ruwords['ружьё'],
    'guns':   ruwords['ружья'],
    'cup':   ruwords['чашка'],
    'cups':   ruwords['чашки'],
}

In [32]:
dict_numeral={
    'one':   r_odin(),
    'three': r_tri(),
}

In [33]:
isinstance(dict_numeral['one'],StNum)

True

## Паттерны

In [34]:
@debug_pp
def p_numeral(s,p):
    return fromdict(dict_numeral)(s,p)

In [35]:
#2->
@debug_pp
def p_adj(s,p):
    return fromdict(dict_adj)(s,p)

In [36]:
@debug_pp
def p_adj_noun3(s,p):
    rezs=[]
    if 1: #исключение словосочетания
        def lp_an_a(str,pos):
            lrezs=word('an')(str,pos)
            if len(lrezs)>0 : return lrezs
            return word('a')(str,pos)
        rezs+=p_seq(s,p,[ lp_an_a, p_noun3 ],r_a_noun)
        #return p_seq(s,p,[ p_adj, p_nekotoryj_noun ],r_adj_noun)
    if len(rezs)>0 : return rezs
    #словосочетание
    return p_seq(s,p,[ p_adj, p_noun3 ],r_adj_noun)    
    
@debug_pp
def p_noun3(s,p):
    rezs=  p_adj_noun3(s,p)
    rezs+= p_numeral(s,p)
    rezs+= fromdict(dict_noun)(s,p)
    return rezs

@debug_pp
def p_noun2(s,p):
    rezs=  p_seq(s,p,[ p_noun3, p_numeral ], r_noun_numeral)
    rezs+= p_noun3(s,p)
    return rezs

@debug_pp
def p_noun1(s,p):
    rezs=  p_seq(s,p,[ p_numeral, p_noun2 ], r_numeral_noun)
    rezs+= p_noun2(s,p)
    return rezs

#1->
@debug_pp
def p_noun(s,p):
    def p_noun1_and_noun(s,p):
        rezs=  p_seq(s,p,[ p_noun1, word('and'), p_noun ],r_noun_and_noun  ,[0,2])
        rezs+= p_seq(s,p,[ p_noun1, word(',')  , p_noun ],r_noun_comma_noun,[0,2])
        return rezs
    rezs=  p_noun1_and_noun(s,p)
    rezs+= p_noun1(s,p)
    return rezs

# Тесты

```
предложения, абзацы
...
нужен LR-парсер, причем недетерминированный
сообщения об ошибках, если ничего не прочитано
```

In [37]:
def tokenize(s) : return [i for i in tokenizer(s)]

In [38]:
p_noun(tokenize('cat'),0)

[(1,
  {type:noun,odush:True,rod:g,chis:ed,pad:ip,word:кошка,och:кошки,talk:[('main', 'кошка')]})]

In [57]:
def decline(s):
    s=[ i for i in tokenizer(s)]
    # добавить дочитывание точки и остаточных пробелов
    rezs=[res for pos,res in p_noun(s,0) if pos==len(s)]
    if len(rezs)!=1:
        raise TextError(rezs)
    tmp=rezs[0]
    print(str(tmp)); #print(repr(tmp))
    #assert isinstance(tmp,Struct) , 'tmp'
    rez=deepcopy(tmp); 
    #assert isinstance(rez,StNum) , 'rez'
    rez.pad='rp'; print('нет',str(rez)); #print(repr(rez))
    rez=deepcopy(tmp); rez.pad='dp'; print('дать',str(rez)); #print(repr(rez))
    rez=deepcopy(tmp); rez.pad='vp'; print('вижу',str(rez)); #print(repr(rez))
    rez=deepcopy(tmp); rez.pad='tp'; print('творю',str(rez)); #print(repr(rez))
    rez=deepcopy(tmp); rez.pad='pp'; print('думаю о',str(rez)); #print(repr(rez))
    

In [40]:
decline('three guns')

три ружья
нет трёх ружей
дать трём ружьям
вижу три ружья
творю тремя ружьями
думаю о трёх ружьях


## Lesson 1

In [41]:
en2ru('cat')

['кошка']

In [42]:
en2ru('a cat')

['кошка']

In [43]:
en2ru('rat')

['крыса']

In [44]:
en2ru('bat')

['летучая мышь']

In [45]:
en2ru('a')

['']

In [46]:
en2ru('lesson')

['урок']

In [47]:
en2ru('a rat, a bat, a cat')

['крыса, летучая мышь, кошка']

## Lesson 2

In [48]:
en2ru('a pen and a hen')

['ручка и курица']

In [49]:
en2ru('a bat and a cat')

['летучая мышь и кошка']

In [50]:
en2ru('a hen, a bat, a cat and a rat')

['курица, летучая мышь, кошка и крыса']

## Lesson 3

In [51]:
en2ru('one')

['один']

In [52]:
en2ru('a bat, a dog and a pig')

['летучая мышь, собака и свинья']

In [53]:
en2ru('one dog')

['одна собака']

In [54]:
en2ru('three dogs')

['три собаки']

In [55]:
en2ru('three bats and one dog')

['три летучие мыши и одна собака']

In [58]:
decline('lesson one')

урок один
нет урока один
дать уроку один
вижу урок один
творю уроком один
думаю о уроке один


## Lesson 4

In [ ]:
en2ru('')

In [ ]:
en2ru('')